In [1]:
import sys
sys.path.append("../../..")
import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
%load_ext autoreload
%autoreload 1
import lightning.pytorch as pl
from lightning.pytorch.callbacks import DeviceStatsMonitor
from lightning.pytorch.tuner.tuning import Tuner
import numpy as np

from ntupleReaders.clue_ntuple_reader import ClueNtupleReader

from ml.dynamic_reduction_network import DynamicReductionNetwork
%aimport ml.regression.drn.modules
%aimport ml.regression.drn.dataset_making
from ml.regression.drn.modules import *
from ml.regression.drn.dataset_making import *
%aimport ml.regression.drn.callbacks
from ml.regression.drn.callbacks import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
reader = ClueNtupleReader("v40", "cmssw", "sim_proton_v46_patchMIP")
datamodule = DRNDataModule(reader, datasetComputationClass=LayerClustersTensorMaker, transformFct=None, 
                multiprocess_loader=False, keepOnGpu="cuda:3"
                )
model = DRNModule(drn=DynamicReductionNetwork(input_dim=4,hidden_dim=20,k=10,output_dim=1,norm=torch.tensor([ 1., 1., 1., 1.])),
    loss=RatioCorrectedLoss([-0.2597882 , -0.24326517,  1.01537901]), optimizer=torch.optim.AdamW)
devices = [3]

In [11]:
trainer = pl.Trainer(accelerator="gpu", devices=devices, enable_progress_bar=True, fast_dev_run=False, logger=True,
                     limit_train_batches=0.1, limit_test_batches=0.1, limit_predict_batches=0.1, limit_val_batches=0.5,
        max_epochs=2,# profiler="simple"
        #callbacks=[SigmaOverECallback("full", every_n_epochs=1)]
             )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model=model, datamodule=datamodule,)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type                    | Params
-------------------------------------------------
0 | drn  | DynamicReductionNetwork | 9.0 K 
-------------------------------------------------
9.0 K     Trainable params
0         Non-trainable params
9.0 K     Total params
0.036     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [14]:
trainer.log_dir

'/grid_mnt/vol_home/llr/cms/cuisset/hgcal/testbeam18/clue3d-dev/src/Plotting/ml/regression/drn/lightning_logs/version_32'

In [15]:
model._hparams

"drn.aggr":       add
"drn.dropout":    0.2
"drn.hidden_dim": 20
"drn.k":          10
"drn.norm_0":     1.0
"drn.norm_1":     1.0
"drn.norm_2":     1.0
"drn.norm_3":     1.0